# Effects of Geopolitical events on currency exchange rates

### **Authors**

* Sai Vamshi Challamala , SaiVamshi.Challamalla@colorado.edu
* Sneha Sasanapuri , snsa4676@colorado.edu
* Amulya Ambati , Ambati.Amulya@colorado.edu
* Lakshmi Prasanna Poluru , Prasanna.Poluru@colorado.edu
* Venkata Harshith Nikhil , VenkataHarshithNikhil.Samudrala@colorado.edu
* Abhishek Reddy Gorla , AbhishekReddy.Gorla@colorado.edu

1. The online published report is at [link](https://amulyaambati.github.io/DSAF-Domain-project/)
2. Link to Github page is at [link](https://github.com/amulyaambati/DSAF-Domain-project)

###**Introduction**

The intricate tapestry of the global financial landscape is woven with complex threads, where seismic events possess the potential to profoundly alter the terrain of currency exchange rates. Few episodes in financial history have left as deep an imprint as the 2008 financial crisis and the convergence of geopolitical tensions during the Russia-Ukraine conflict. Omitting intricate specifics, our analysis explores the dynamic interaction between geopolitical tension and global financial instability.

**Goal of the Project**

Can we analyse the effect of geopolitical tension or natural events on fluctuations of currency exchange rates?


**Exploring global currency exchange rates dataset**

*The Significance of Currency Exchange Rates*

Currency exchange rates occupy a pivotal role within the global economic landscape, exerting substantial influence over matters of trade, investment, and international financial transactions. These rates reflect the relative values of different currencies and have a profound impact on businesses, governments, investors, and individuals alike. Understanding and analyzing currency exchange rate data is essential for making informed decisions in the realm of international finance.

In this report, we delve into the world of exploratory data analysis of currency exchange rates spanning the period from 2005 to 2023. This introduction serves as a primer for the subsequent sections, providing an overview of why currency exchange rate data is crucial, its sources, and how it can be analyzed to gain valuable insights.

*Description*

The currency exchange rates in this dataset represent the value of one currency in terms of US Dollar for every single day. For example, if the exchange rate between the U.S. Dollar (USD) and the Euro (EUR) is 1.20 on the date Febuary 19,2018 , it means that one USD can be exchanged for 1.20 EUR on that particular day. On a whole it's a day level dataset.

*Source*

Data Source: The data employed in this report has been sourced from an external Application Programming Interface (API) accessible at https://api.exchangerate.host/timeseries, as of the date September 24, 2023. This particular API was chosen due to its extensive day level information, making it an ideal source for our analysis of currency exchange rate trends.

*Retrieval*

Data retrieval from the API was executed through the utilization of HTTP GET requests directed to designated endpoints. This procedure was accomplished through the Python programming language, employing the 'requests' library, a well-regarded tool in the field of data acquisition.



**Install Required Libraries:**

Let's begin by importing some useful libraries, including _"pandas", "numpy", "datetime", and "plotly"_. These libraries will enable us to perform data manipulation, address data quality issues, and create compelling visualizations.

In [1]:
import pandas as pd
from datetime import date,timedelta,datetime
import requests
from io import StringIO
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Data pull


Recognizing the significance of historical data spanning the preceding two decades, we have initiated this data acquisition process begining  from the year 2000 to the present date (09/23/2023)'.



NOTE : Unfortunately the API has suddenly started asking for api_key since 09/28/2023. Hence commented the below data pulling code and loading the data from csv file (that was saved as a backup after pulling from API).

In [2]:
def get_rates(base_currency,start_date,end_date):
  url = f'https://api.exchangerate.host/timeseries'
  payload = {'base':base_currency,'start_date':start_date,'end_date':end_date}
  response = requests.get(url,params= payload)
  data  = response.json()
  return data

The `get_rates()` function returns a dictionary with data starting from `start_date` to `end_date`, where the key `rates` is of our interest, which contains the data in the following format :

_'rates' : {'2000-01-01' :{'usd':1 , 'inr' : 45, 'yun' : ......}, '2000-01-02' :{'usd':1 , 'inr' : 45.1, 'yun' : ......} .................'2023-09-01' :{'usd':1 , 'inr' : 82, 'yun' : ......}}_


In [3]:
# history = []
# for year in range(2000,2024):
#   start_date = date(year, 1, 1)
#   start_year= start_date.year
#   if start_year%4==0:
#     end_date = start_date + timedelta(days = 365)
#   else:
#     end_date = start_date + timedelta(days = 364)
#   rates_data = get_rates('USD',start_date,end_date)
#   for date_ in rates_data['rates'].keys():
#     for currency_code,value in rates_data['rates'][date_].items():
#       history.append([date_,currency_code,value])

While trying to pull the data from the above designated API, we found a limitation pertaining to data retrieval capacity within a single API call. So, in order to tackle this issue our solution entails invoking the API within a loop, specifically targeting each year.


In an effort to circumvent data loss during a leap year, we have implemented a condition to ascertain the leap year status, subsequently adjusting the number of days as necessitated within the code excerpt provided below

In [4]:
# df = pd.DataFrame(history, columns=['date','currency_code','exchange_rate'])
# df['year'] = pd.to_datetime(df['date']).dt.year

In this stage of our process, we transformed previously acquired data, which exists in a list format, into a structured pandas dataframe. Additionally, we have extracted year data from date column and stored in a sepearte columnn so that any analysis and aggreagtions can be done at an yearly level.

In [5]:
#df.to_csv('rates.csv')

Finally, the gathered data is stored in a csv file, so as to avoid re-running the data pull multiple times and to have a backup in case API stops working.

Uploaded the dataset to github and reading it from there, so as to make the report reproducible.

In [6]:
url = 'https://raw.githubusercontent.com/amulyaambati/DSAF-Domain-project/main/rates.csv'
response = requests.get(url)
data = StringIO(response.text)
df = pd.read_csv(data)

In [7]:
df.head(2)

,date,currency_code,exchange_rate,year
0,1/1/2000,USD,1.000000,2000
1,1/1/2000,JPY,102.259606,2000


The above exchange rates dataset contains the data of all the existing currencies. Since its not possible to analyze all the currency exchange rates, we want to focus on currencies of particular countries of interest. Hence for that reason we would need a `country to currency code` mapping.

In [8]:
url = 'https://raw.githubusercontent.com/amulyaambati/DSAF-Domain-project/main/country_code_to_currency_code.csv'
response = requests.get(url)
data = StringIO(response.text)
codes = pd.read_csv(data)
codes =  codes[['Currency_Code','Country','Currency Name']]
codes.rename(columns={'Currency_Code':'currency_code'},inplace=True)
codes.head()

,currency_code,Country,Currency Name
0,AFN,AFGHANISTAN,Afghani
1,ALL,ALBANIA,Lek
2,DZD,ALGERIA,Algerian Dinar
3,USD,AMERICAN SAMOA,US Dollar
4,EUR,ANDORRA,Euro


In the next step, we are integrating currency exchange rate data with corresponding country currency information. Given our specific focus on particular countries of interest, the inclusion of currency-to-country mapping is indispensable for meaningful analysis. As such, we have employed an 'inner' join operation to combine these datasets, ensuring that only records with matching currency-to-country information are retained for further analysis.

In [9]:
df = df.merge(codes,how='inner',on='currency_code')

In [10]:
df.shape

(1644353, 6)

In [11]:
df.head()

,date,currency_code,exchange_rate,year,Country,Currency Name
0,1/1/2000,USD,1.0,2000,AMERICAN SAMOA,US Dollar
1,1/1/2000,USD,1.0,2000,BONAIRE| SINT EUSTATIUS AND SABA,US Dollar
2,1/1/2000,USD,1.0,2000,BRITISH INDIAN OCEAN TERRITORY (THE),US Dollar
3,1/1/2000,USD,1.0,2000,ECUADOR,US Dollar
4,1/1/2000,USD,1.0,2000,EL SALVADOR,US Dollar


### End of data extraction



We have identified the top 20 GDP countries by referring to the information available at [link](https://www.worldometers.info/gdp/gdp-by-country/), and have created a list of these nations.  


In [12]:
top20_countries = ['CHINA',
                   'JAPAN',
                   'GERMANY',
                   'INDIA',
                   'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)',
                   'FRANCE',
                   'RUSSIAN FEDERATION (THE)',
                   'CANADA',
                   'ITALY',
                   'BRAZIL',
                   'AUSTRALIA',
                   'KOREA (THE REPUBLIC OF)',
                   'MEXICO',
                   'SPAIN',
                   'INDONESIA',
                   'SAUDI ARABIA',
                   'NETHERLANDS (THE)',
                   'TURKEY',
                   'SWITZERLAND',
                   'POLAND']

Here we are subsetting the data for above countries.

In [13]:
sub_df = df[df['Country'].isin(top20_countries)]

In [14]:
sub_df.shape

(160946, 6)

## Data Quality check

### Checking for missing entries/values.

In this context, we are verifying the presence of exchange rate values for every date between January 1, 2000, and September 23, 2023.

In [15]:
date1 = date(2000, 1, 1)
date2 = date(2023, 9, 23)
print((date2 - date1).days + 1, "days")

8667 days


In [16]:
sub_df[['Country','date']].drop_duplicates()['Country'].value_counts()

JAPAN                                                         8667
FRANCE                                                        8667
SPAIN                                                         8667
UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)    8667
ITALY                                                         8667
GERMANY                                                       8667
NETHERLANDS (THE)                                             8667
KOREA (THE REPUBLIC OF)                                       8667
CANADA                                                        8667
AUSTRALIA                                                     8667
SWITZERLAND                                                   8667
POLAND                                                        8667
INDONESIA                                                     7229
CHINA                                                         7225
MEXICO                                                        

We are seeing that for few countries, no.of exchange rate entries is lesser in number. Hence, it is important to see if there is any pattern in missing entries.

In [17]:
pd.pivot_table(data = sub_df , index = 'Country',columns = 'year',aggfunc='count')

Currency Name         \
year                                                        2000   2001   
Country                                                                   
AUSTRALIA                                                  366.0  365.0   
BRAZIL                                                       NaN    NaN   
CANADA                                                     366.0  365.0   
CHINA                                                        NaN    NaN   
FRANCE                                                     366.0  365.0   
GERMANY                                                    366.0  365.0   
INDIA                                                        NaN    NaN   
INDONESIA                                                    NaN    NaN   
ITALY                                                      366.0  365.0   
JAPAN                                                      366.0  365.0   
KOREA (THE REPUBLIC OF)                                    366.0  365.0   
MEXICO                                                       NaN    NaN   
NETHERLANDS (THE)                                          366.0  365.0   
POLAND                                                     366.0  365.0   
RUSSIAN FEDERATION (THE)                                     NaN    NaN   
SAUDI ARABIA                                                 NaN    NaN   
SPAIN                                                      366.0  365.0   
SWITZERLAND                                                366.0  365.0   
TURKEY                                                       NaN    NaN   
UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IR...         366.0  365.0   

                                                                         \
year                                                 2002   2003   2004   
Country                                                                   
AUSTRALIA                                           365.0  365.0  366.0   
BRAZIL                                                NaN   26.0  355.0   
CANADA                                              365.0  365.0  366.0   
CHINA                                                 NaN   26.0  359.0   
FRANCE                                              365.0  365.0  366.0   
GERMANY                                             365.0  365.0  366.0   
INDIA                                                 NaN   31.0  364.0   
INDONESIA                                             NaN   31.0  358.0   
ITALY                                               365.0  365.0  366.0   
JAPAN                                               365.0  365.0  366.0   
KOREA (THE REPUBLIC OF)                             365.0  365.0  366.0   
MEXICO                                                NaN   31.0  362.0   
NETHERLANDS (THE)                                   365.0  365.0  366.0   
POLAND                                              365.0  365.0  366.0   
RUSSIAN FEDERATION (THE)                              NaN   31.0   48.0   
SAUDI ARABIA                                          NaN   31.0  358.0   
SPAIN                                               365.0  365.0  366.0   
SWITZERLAND                                         365.0  365.0  366.0   
TURKEY                                                NaN    NaN    NaN   
UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IR...  365.0  365.0  366.0   

                                                                         \
year                                                 2005   2006   2007   
Country                                                                   
AUSTRALIA                                           365.0  365.0  365.0   
BRAZIL                                              358.0  365.0  361.0   
CANADA                                              365.0  365.0  365.0   
CHINA                                               365.0  365.0  365.0   
FRANCE                                              365.0  365.0  365.0   

In [18]:
sub_df[['Country','year']].drop_duplicates()['Country'].value_counts()

JAPAN                                                         24
FRANCE                                                        24
SPAIN                                                         24
UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)    24
ITALY                                                         24
GERMANY                                                       24
NETHERLANDS (THE)                                             24
KOREA (THE REPUBLIC OF)                                       24
CANADA                                                        24
AUSTRALIA                                                     24
SWITZERLAND                                                   24
POLAND                                                        24
BRAZIL                                                        21
CHINA                                                         21
INDONESIA                                                     21
INDIA                    

From the above two outputs it is evident that for few currency rates, data is completely missing from 2000 to 2004.In the subsequent years, few values are missing for some of the currencies. Hence, we will be completely removing the data from 2000 to 2004 and carry out the analysis from 2005 to 2023.

In [19]:
sub_df_1 = sub_df[sub_df['year'] > 2004]

In [20]:
sub_df_1.shape

(136611, 6)

Let's calculate the missing value percentage post 2004, to see how to handle the data.

In [21]:
date1 = date(2005, 1, 1)
date2 = date(2023, 9, 23)
total_days = (date2-date1).days + 1

sub_df_1[['Country']].value_counts()/ total_days

Country                                                   
AUSTRALIA                                                     1.000000
ITALY                                                         1.000000
SWITZERLAND                                                   1.000000
SPAIN                                                         1.000000
POLAND                                                        1.000000
NETHERLANDS (THE)                                             1.000000
JAPAN                                                         1.000000
KOREA (THE REPUBLIC OF)                                       1.000000
INDONESIA                                                     1.000000
GERMANY                                                       1.000000
FRANCE                                                        1.000000
CHINA                                                         1.000000
CANADA                                                        1.000000
UNITED KINGDOM OF 

For 7 countries, less then 0.01% of the data missing, which is very minute.Hence proceeding with the current data.

### Checking for any absurd values

In [22]:
sub_df_1['exchange_rate'].describe()

count    136611.000000
mean        652.812196
std        2597.284434
min           0.474218
25%           0.894055
50%           2.273476
75%          23.548924
max       16502.554580
Name: exchange_rate, dtype: float64

There aren't any non-positive values as such. Hence the data is good to go.

## Data Analysis and Visualization

In [23]:
def get_country_year_stat(df,country,year):
  grouped_data = df.groupby(['Country','year'])['exchange_rate'].describe().reset_index()
  stats = grouped_data[(grouped_data['Country']==country) & (grouped_data['year']==year)].drop(['count','25%','50%','75%'],axis=1)
  return stats

In [24]:
country = 'AUSTRALIA'
year = 2008
get_country_year_stat(sub_df,country,year)

,Country,year,mean,std,min,max
8,AUSTRALIA,2008,1.195436,0.181465,1.021133,1.651584


The next analysis and vizualization that we would like to get is, the general trend of country's currency exchange rate over the years. To get this trend line, we need to aggregate the data at year level through mean.

In [25]:
def get_year_wise(df):
  stats = df.groupby(['Country','year'])['exchange_rate'].mean().reset_index()
  return stats

In [26]:
year_wise = get_year_wise(sub_df_1)

In [27]:
fig = px.line(year_wise, x="year", y="exchange_rate", color='Country',markers= True)
fig.show()

### Decade level %change

For a more comprehensive analysis, we have determined that visualizing the decade-wise percentage change in exchange rates for countries would provide valuable insights. Instead of solely calculating the annual means of exchange rates for countries, we have undertaken the task of extracting decade-wise percentage changes in exchange rates. This approach allows us to discern broader trends and fluctuations over time, enhancing the depth and scope of our analysis.

In [28]:
def decade_wise_change(df,decade_start_date, decade_end_date):
  decade_data = df[(df['date']== decade_start_date) | (df['date']== decade_end_date)]
  decade_data.sort_values(['Currency Name','Country','date'],inplace=True)
  stat = decade_data.groupby(['Currency Name','Country']).apply(lambda x : ((x['exchange_rate'].iloc[-1]-x['exchange_rate'].iloc[0])/x['exchange_rate'].iloc[0]) *100.0).reset_index()
  stat.rename(columns={0:'%change'},inplace=True)
  stat.sort_values('%change',inplace=True)
  return stat

In [29]:
decade_start_date = '1/1/2013'
decade_end_date = '9/1/2023'

decade_level_change =decade_wise_change(sub_df_1,decade_start_date,decade_end_date)

In [30]:
decade_fig = px.bar(decade_level_change, y='Country', x='%change',color='%change',
             title='Decade level(2013 to 2023) %Change in Exchange Rate', orientation='h',width=750, height=750)
decade_fig.show()

From the above graph, we can conclude that in the past decade that is (2013 to 2023), Turkey's currency has the highest depreciation wrt USD, whereas Switzerland's currency is the only currency that has appreciated in the past decade wrt USD.

Fun fact : Saudi Arabia operates under a fixed exchange rate system where the Saudi Riyal is pegged to the US Dollar. Hence there hasn't been much change for the currenies exchange rate.

### Analysis of effect on geopolitical tension or natural events on currenct exchange rates

In the next phase, we would like to analyze and visualize the effect of sesmic events and geopolitical tensions on the exchange rates.


Setting up a generic code, which is able to modify for the data in required format for any given event (by provding the date range).

In [31]:
def get_event_data(df,start_year,end_year,n,level = 'yearly',countries = None):
  if countries is None:
    sub_df = df.copy()
  else:
    sub_df = df[df['Country'].isin(countries)]

  sub_df = sub_df[(sub_df['year']<= end_year+n) & (sub_df['year']>=start_year-n)]
  sub_df['date'] = pd.to_datetime(sub_df['date'],format = '%m/%d/%Y')
  sub_df['half_year'] = sub_df['date'].dt.month // 7

  if level == 'half_yearly':
    grouping = ['Currency Name','Country','year','half_year']
  else:
    grouping = ['Currency Name','Country','year']

  agg_data = sub_df.groupby(grouping).mean().reset_index()
  agg_data.sort_values(grouping,inplace=True)
  agg_data['lag_1'] = agg_data.groupby(['Currency Name','Country'])['exchange_rate'].transform(lambda x : x.shift())

  if level == 'half_yearly':
    agg_data['%change_wrt_previous_halfyear'] = (agg_data['exchange_rate']- agg_data['lag_1'])/agg_data['lag_1']
    agg_data['year_halfyear'] = agg_data['year'].astype(str) + '_' + agg_data['half_year'].astype(str)
  else:
    agg_data['%change_wrt_previous'] = (agg_data['exchange_rate']- agg_data['lag_1'])/agg_data['lag_1']
  return agg_data

### The Global Financial Crisis

The 2008 financial crisis had a significant impact on global currency markets, and the value of the USD experienced fluctuations in response to the evolving economic and financial landscape. It is evident from graph that USD has initially strengthened due to "safe-heaven" sentiment till 2009 and started to depreciate in the aftermath of crisis.

In [32]:
event_df_2008 = get_event_data(sub_df_1,2007,2009,3)

In [33]:
fig = px.line(event_df_2008, x="year", y="%change_wrt_previous", color='Currency Name',markers= True)
fig.show()

### The Covid Pandemic

The COVID-19 pandemic had a profound impact on foreign exchange (forex) rates, leading to significant fluctuations and changes in currency values.

During times of crisis, US-based multinational corporations and investors epatriated funds from abroad. This involves converting foreign currencies into USD, increasing the demand for the US Dollar and contributing to its appreciation in its early stages.

Another interesting insight from the graph is that USD started to depreciate in later stages of COVID. This can be attributed to "Risk Appetite". Improved market sentiment and risk appetite led investors to seek higher-yielding assets in other currencies and markets. As investors moved away from the safety of the USD, it put downward pressure on its value leading to its depeciation.

In [34]:
event_df_covid = get_event_data(sub_df_1,2020,2021,1,level = 'half_yearly')

In [35]:
fig = px.line(event_df_covid, x="year_halfyear", y="%change_wrt_previous_halfyear", color='Currency Name',markers= True)
fig.show()

### Russia crisis with Ukraine

In response to Russia's actions in Ukraine, the US and EU imposed a series of economic sanctions on Russia. These sanctions targeted Russian economic, financial and energy sectors. Sanctions restricted Russia's access to international capital markets and led to paraount dip in their exchange rate.

In [36]:
event_df_russia = get_event_data(sub_df_1,2022,2023,1,level = 'half_yearly',countries =['RUSSIAN FEDERATION (THE)'] )

In [37]:
fig = px.line(event_df_russia, x="year_halfyear", y="%change_wrt_previous_halfyear", color='Currency Name',markers= True,width=750, height=750)
fig.show()

### Conclusion:

It's important to note that currency markets are influenced by a complex interplay of factors, and exchange rates can be influenced by a combination of economic, fiscal policy, and market sentiment considerations. The insights gained through this analysis of currency exchange rates during the econoimc turmoil instances stands as an approximate indicator of their financial health. If other factros affecting exchange rates were also considered for analysis, it would be possible to accurately predict the economic health.

